In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
import sys
sys.path.append('../../')

import os
import pandas as pd
import numpy as np
import config
import utils
import torch
import pickle
import train_impf_MLP

from Model import Impf_DNAMLP
from torch.nn.functional import softmax
from utils import impf_make_ndarray_from_csv, get_int_label
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from Dataset import CNS

In [3]:
print(utils.low_perf_groups)

['Glio-neuronal', 'Other glioma']


In [4]:
inner_fold = '3.4'
outer_fold = f"{inner_fold.split('.')[0]}.0"
group = 'Sella'
alg = 'MLP'
print(outer_fold)

3.0


In [5]:
dat_cfg = config.data_config
clf_cfg = config.classifier_config
impf_cfg = config.impf_config
TRAIN_CSV_DIR = dat_cfg['TRAIN_CSV_DIR']
TEST_CSV_DIR = dat_cfg['TEST_CSV_DIR']

INNER_FOLD_CSV = os.path.join(TEST_CSV_DIR, group, f'{inner_fold}_test.csv')
OUTER_FOLD_CSV = os.path.join(TEST_CSV_DIR, group, f'{outer_fold}_test.csv')
device = impf_cfg['device']
device

'cuda'

In [6]:
with open(os.path.join(impf_cfg['IMPORTANT_FEATURES_DIR'], alg, group, f'{outer_fold}_combined.pkl'), 'rb') as handle:
    impf = pickle.load(handle)
print(len(impf))

2132


In [7]:
impf

{'cg03783663',
 'cg26923862',
 'cg13894852',
 'cg02772995',
 'cg00926215',
 'cg22955387',
 'cg21518089',
 'cg05257898',
 'cg03791955',
 'cg03604774',
 'cg26783057',
 'cg06694734',
 'cg06059483',
 'cg03790988',
 'cg00872984',
 'cg00998387',
 'cg07153588',
 'cg08684879',
 'cg04790977',
 'cg06668065',
 'cg05349016',
 'cg06398242',
 'cg02555243',
 'cg08469845',
 'cg22055891',
 'cg17861863',
 'cg04227631',
 'cg16806891',
 'cg08419873',
 'cg13683667',
 'cg13909661',
 'cg21597595',
 'cg25168494',
 'cg26680644',
 'cg03668763',
 'cg25757115',
 'cg25233139',
 'cg08605301',
 'cg16405960',
 'cg26361533',
 'cg04552625',
 'cg22198853',
 'cg05396243',
 'cg04703620',
 'cg01652021',
 'cg25416067',
 'cg03799024',
 'cg00590260',
 'cg09411910',
 'cg06821828',
 'cg09480190',
 'cg26819695',
 'cg05850327',
 'cg02892357',
 'cg08230177',
 'cg08482837',
 'cg03242698',
 'cg20583060',
 'cg25783987',
 'cg03425412',
 'cg18422058',
 'cg14796655',
 'cg23683800',
 'cg05357932',
 'cg23976934',
 'cg13392078',
 'cg088476

In [8]:
df_inner = pd.read_csv(INNER_FOLD_CSV, index_col=0)
df_outer = pd.read_csv(OUTER_FOLD_CSV, index_col=0)
df_inner.shape, df_outer.shape

((42, 10002), (59, 10002))

In [57]:
pd.Series(pd.read_csv(OUTER_FOLD_CSV, index_col=0).iloc[:, -1]).value_counts()

Sella      34
Control    25
Name: class_group, dtype: int64

## Load random test sample and predict

In [58]:
rand = np.random.randint(df_outer.shape[0])
sample = df_outer.iloc[rand, :]
rand, sample

(13,
 cg22054918     0.037249
 cg16476975     0.028846
 cg25570913     0.250055
 cg03940848     0.037043
 cg02809746     0.130886
                  ...   
 cg22513691     0.855403
 cg00743629     0.467758
 cg13621317     0.862883
 label          CPH, PAP
 class_group       Sella
 Name: 9305651037_R06C01, Length: 10002, dtype: object)

In [59]:
sample = df_outer.loc[:, sorted(list(impf))].iloc[rand,:]
label = get_int_label(df_outer.iloc[rand, -1], group)
sample, sample.shape, label

(cg00000165    0.385539
 cg00011740    0.129036
 cg00017157    0.644799
 cg00021256    0.428564
 cg00025439    0.550269
                 ...   
 cg27551227    0.363038
 cg27579771    0.374261
 cg27596415    0.116872
 cg27640763    0.085532
 cg27640794    0.135263
 Name: 9305651037_R06C01, Length: 2132, dtype: float64,
 (2132,),
 0)

In [60]:
sample_features = np.expand_dims(np.array(sample), axis = 0)
sample_features
# sample_features, type(sample_features), sample_features.shape, type(sample_features[0][0])

array([[0.38553932, 0.12903609, 0.64479892, ..., 0.11687154, 0.08553154,
        0.13526285]])

In [61]:
IMPF_MLP_STATE_FILE = os.path.join(impf_cfg['MLP_BEST_STATES_DIR'], alg, group, f'{inner_fold}.pth')

n_classes = impf_cfg['n_classes']
in_features = len(impf)

model = Impf_DNAMLP(in_features, n_classes)
if torch.cuda.is_available():
    model.load_state_dict(torch.load(IMPF_MLP_STATE_FILE))
else:
    model.load_state_dict(torch.load(IMPF_MLP_STATE_FILE, map_location='cpu'))
model.to(device)

Impf_DNAMLP(
  (densenet): Sequential(
    (0): Linear(in_features=2132, out_features=2, bias=True)
  )
)

In [62]:
sample_features.shape

(1, 2132)

In [63]:
probs = softmax(model((torch.Tensor(sample_features).to(device))), dim = 1)
probs

tensor([[0.7744, 0.2256]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [64]:
# epoch = 1
# test_features, test_labels = impf_make_ndarray_from_csv(group, outer_fold, impf, mode = 'test')
# test_labels_int = np.array([get_int_label(label, group) for label in test_labels])
# test_dataset = CNS(test_features, test_labels_int, mode = 'val')
# test_loader = DataLoader(test_dataset, batch_size = impf_cfg['mlp_val_batch_size'], shuffle = False)
# criterion = CrossEntropyLoss()
# optimizer = Adam(model.parameters(), lr = impf_cfg['mlp_lr'], weight_decay = impf_cfg['mlp_weight_decay'])
# res = train_impf_MLP.impf_val_epoch(epoch, model, test_loader, criterion, impf_cfg['device'])
# test_loss, test_acc, test_me, test_bs, test_auc, test_f1, test_precision, test_recall, test_cfs = res

In [65]:
# res

In [66]:
# train_features, train_labels = impf_make_ndarray_from_csv(group, inner_fold, impf, mode = 'train')
# val_features, val_labels = impf_make_ndarray_from_csv(group, inner_fold, impf, mode = 'test')
# test_features, test_labels = impf_make_ndarray_from_csv(group, outer_fold, impf, mode = 'test')

# train_labels_int = np.array([get_int_label(label, group) for label in train_labels])
# val_labels_int = np.array([get_int_label(label, group) for label in val_labels])
# test_labels_int = np.array([get_int_label(label, group) for label in test_labels])

# train_dataset = CNS(train_features, train_labels_int, mode = 'train')
# val_dataset = CNS(val_features, val_labels_int, mode = 'val')
# test_dataset = CNS(test_features, test_labels_int, mode = 'val')
# train_loader = DataLoader(train_dataset, batch_size = impf_cfg['mlp_train_batch_size'], shuffle = True)
# val_loader = DataLoader(val_dataset, batch_size = impf_cfg['mlp_val_batch_size'], shuffle = False)
# test_loader = DataLoader(test_dataset, batch_size = impf_cfg['mlp_val_batch_size'], shuffle = False)

# # Init model object
# in_features = len(impf)
# if group in utils.low_perf_groups:
#     model = Impf_GlioMLP(in_features, impf_cfg['n_classes'])
# else:
#     model = Impf_DNAMLP(in_features, impf_cfg['n_classes'])

In [67]:
# %%capture
# all_res = {}
# for fold in utils.inner_folds:
#     res = train_impf_MLP.impf_run(group, alg, fold, train_loader, val_loader, test_loader, model, criterion, optimizer, impf_cfg, 'no_save')
#     all_res[f'{fold}'] = res

In [68]:
# all_res